In [3]:
nums = [1, 2, 3, 4]

for num in nums:
    print(num)

print('---')

index = 0
while True:
    try:
        num = nums.__getitem__(index)
        print(num)
        index += 1
    except IndexError:
        break


1
2
3
4
---
1
2
3
4


In [18]:
class Sequence:
    def __init__(self, length):
        self.length = length

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        if index >= len(self) or index < -len(self):
            raise IndexError
        return index


s = Sequence(5)

print(s[0])
# print(s[100]) --> IndexError
# print(s[-100]) --> IndexError

print('---')

for index in s:
    print(index)

print('---')

for index in s:
    print(index)


0
---
0
1
2
3
4
---
0
1
2
3
4


In [15]:
print(type(slice(None, None, -1)))
print(slice(0, 5).indices(3))
print(slice(0, 5).indices(8))
print(type(slice(0, 5).indices(8)))

print(list(range(*slice(0, 5).indices(8))))


<class 'slice'>
(0, 3, 1)
(0, 5, 1)
<class 'tuple'>
[0, 1, 2, 3, 4]


In [18]:
from functools import lru_cache


class Fib:
    def __init__(self, n):
        self.n = n

    @staticmethod
    @lru_cache(2 ** 10)
    def fib(n):
        if n < 2: return 1
        return Fib.fib(n - 1) + Fib.fib(n - 2)

    def __len__(self):
        return self.n
    
    def __getitem__(self, index):
        if isinstance(index, int):
            if index < 0:
                index += len(self)
            if index < 0 or index >= len(self):
                raise IndexError
            return Fib.fib(index)
        else:
            # slice.indicies(length) --> tuple
            # list(range(tuple))
            return [Fib.fib(value) for value in range(*index.indices(len(self)))]
    

f = Fib(10)

print(list(f))
print(f[-1])

print(f[0:5])

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
55
[1, 1, 2, 3, 5]


In [19]:
class Person:
    def __init__(self, name):
        self.name = name

    def __add__(self, other):
        print('__add__')
        print(other.name)

    def __radd__(self, other):
        return self.__add__(other)
    
    def __iadd__(self, other):
        print('__iadd__')
        print(other.name)
        return self


class OtherPerson:
    def __init__(self, name):
        self.name = name


p = Person('Ajaxander')
p2 = OtherPerson('Jesmike')

p += p2

print(p)

__iadd__
Jesmike


In [24]:
class Person:
    def __init__(self, name):
        self.name = name

    def __add__(self, other):
        return self.__class__(self.name + other.name)
    
    def __iadd__(self, other):
        self.name += other.name
        return self
    
    def __radd__(self, other):
        return self.__add__(other)
    
    def __contains__(self, value):
        return value in self.name
    

p1 = Person('Ajaxander')
p2 = Person('Jesmike')

print((p1 + p2).name)

print(hex(id(p1)))

p1 += p2

print(hex(id(p1)))

print(p1.name)

print('A' in p1)

AjaxanderJesmike
0x1046f6860
0x1046f6860
AjaxanderJesmike
True


In [17]:
# Custom Sequences - Part 2B

import numbers


class Point:
    def __init__(self, x, y):
        if isinstance(x, numbers.Real) and isinstance(y, numbers.Real):
            self._pt = (x, y)
        else:
            raise TypeError('Point co-ordinates must be real numbers.')
        
    def __repr__(self):
        return f'Point(x={self._pt[0]}, y={self._pt[1]})'
    
    def __len__(self):
        return len(self._pt)
    
    def __getitem__(self, index):
        # if index >= len(self):
        #     raise IndexError
        # index == 2 --> tuple self._pt raise IndexError
        return self._pt[index]
    

class Polygon:
    def __init__(self, *pts):
        if pts:
            self._pts = [Point(*pt) for pt in pts]
        else:
            self._pts = []

    def __repr__(self):
        points = ', '.join(str(pt) for pt in self._pts)
        return f'Polygon({points})'
    
    def __len__(self):
        return len(self._pts)
    
    def __getitem__(self, index):
        return self._pts[index]
    
    def __add__(self, other):
        if isinstance(other, Polygon):
            return Polygon(*self._pts, *other._pts)
        else:
            raise TypeError
        
    def __iadd__(self, other):
        if isinstance(other, Polygon):
            # self._pts += other._pts
            self._pts = self._pts + other._pts
            return self
        else:
            raise TypeError
        
    def append(self, pt):
        self._pts.append(Point(*pt))

    def insert(self, index, pt):
        self._pts.insert(index, Point(*pt))

    def __setitem__(self, s, values):
        try:
            self._pts[s] = Point(*values) if isinstance(s, int) else \
                [Point(*pt) for pt in values]
        except TypeError:
            raise TypeError('Uncompatible!')
        
    def __delitem__(self, s):
        del self._pts[s]

    def pop(self, s):
        return self._pts.pop(s)
    

p = Polygon((0, 0), Point(1, 1))

# print(p) Polygon(Point(x=0, y=0), Point(x=1, y=1))
# print( p[::-1]) [Point(x=1, y=1), Point(x=0, y=0)]
